In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only, encoder_with_convs_and_symmetry_new
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tflearn.layers.conv import avg_pool_1d
from pc_completions.evaluation import basic_comletion_measures

from tflearn.layers.conv import conv_1d
from tflearn.layers.normalization import batch_normalization
from testing_encoder_decoder import decoder_with_convs_only

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_points = 2048
top_data_dir = '/orions4-zfs/projects/optas/DATA'
top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))

In [5]:
do_training = True
load_pretrained_model = False
load_epoch = None
random_seed = 42
loss = 'chamfer'
training_epochs = 250
batch_size = 50
learning_rate = 0.0005

In [6]:
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt')
val_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt')
test_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt')

splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)
tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
val_files = splitter.load_splits(val_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(val_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
test_files = splitter.load_splits(test_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(test_files, n_threads=20, loader=snc_loader, verbose=True)
test_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.


In [11]:
b_norm = True
encoder_args = {'n_filters': [64, 128, 128, 256],
                'filter_sizes': [9, 3, 2, 2],
                'strides': [1, 1, 1, 1],
                'b_norm': b_norm,
                'pool': avg_pool_1d,
                'pool_sizes': [2, None, 2, None],
                }


decoder_args = {'layer_sizes': [6, 6, 12, 12],
                'b_norm': b_norm,
                'non_linearity': None,
                }

In [ ]:
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings/temp')

conf = Conf(
            n_input = [n_pc_points, 3],
            loss = loss,
            training_epochs = training_epochs,
            batch_size = batch_size,
            denoising = False,
            learning_rate = learning_rate,
            train_dir = train_dir,             
            loss_display_step = 1,
            saver_step = None,
            z_rotate = False,
            encoder = encoder_with_convs_and_symmetry_new, 
            decoder = decoder_with_fc_only,
            encoder_args = encoder_args,
            decoder_args = decoder_args
           )

conf.experiment_name = 'Test'
conf.held_out_step = 5
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

print ae.trainable_parameters()

if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout, held_out_data=val_data)
    fout.close()

conv params =  256
bnorm params =  128
Tensor("Test_1/Relu:0", shape=(?, 2048, 64), dtype=float32) 131072
conv params =  8320
bnorm params =  256
Tensor("Test_1/Relu_1:0", shape=(?, 2048, 128), dtype=float32) 262144
conv params =  16512
bnorm params =  256
Tensor("Test_1/Relu_2:0", shape=(?, 2048, 128), dtype=float32) 262144
conv params =  33024
bnorm params =  512
Tensor("Test_1/Relu_3:0", shape=(?, 2048, 256), dtype=float32) 524288
Tensor("Test_1/Max:0", shape=(?, 256), dtype=float32) 256
7016832
('Epoch:', '0001', 'training time (minutes)=', '0.1023', 'loss=', '0.018783161')
('Epoch:', '0002', 'training time (minutes)=', '0.0944', 'loss=', '0.006692419')
('Epoch:', '0003', 'training time (minutes)=', '0.0948', 'loss=', '0.006328637')
('Epoch:', '0004', 'training time (minutes)=', '0.0944', 'loss=', '0.006023390')
('Epoch:', '0005', 'training time (minutes)=', '0.0942', 'loss=', '0.005691267')
('Held Out Data :', 'forward time (minutes)=', '0.0017', 'loss=', '0.005136445')
('Epoch:',

In [10]:
ae.trainable_parameters()

131028